# Inference
Creating a conversion function is one thing, applying it is another challange.
We can tell when there is sunlight, and assume the LEDs are only turned on at night. 
In practise however, we also commonly see LED lighting being used below x umol of sunlight.
This is a tricky situation as the amounth of LED and Sun light varies, making it tricky to correct. 
Still a combination between the two claibration could be used, as something is better than nothing


In [16]:
import pandas as pd
import plotly.graph_objects as go

In [17]:
sens_df = pd.read_csv('data/sensor_data.csv', sep = ';')

# Convert date to datetime
sens_df['date'] = pd.to_datetime(sens_df['date'], format='%H:%M:%S')
sens_df['Date'] = sens_df['date'].dt.date
sens_df['Time'] = sens_df['date'].dt.time

# Convert to floats, excel uses commas instead of dots
sens_df['Counts'] = sens_df['Counts'].str.replace(',', '.').astype(float)

In [18]:
#Conversions
# LED
led_A = 55.14
led_B = 5.89

# SUN
sun_A = 33.2
sun_B = 1.2

In [29]:
# Sun rise and sundown times downloaded from: https://www.suntoday.org/sunrise-sunset/2021.html
# For Wageningen, Netherlands
sunrise_down_df = pd.read_csv("data/sunrise_sundown.csv", sep = ';')

# The 'lookup_daynight' assumes sunrise is column 1 and sundows is column 2! 

# Set column types correct
sunrise_down_df['Date'] = pd.to_datetime(sunrise_down_df['Date'], format='%d-%m-%Y')
sunrise_down_df['Date'] = sunrise_down_df['Date'].dt.date
sunrise_down_df['sunrise'] = pd.to_datetime(sunrise_down_df['sunrise'], format="%H:%M:%S").dt.time
sunrise_down_df['sundown'] = pd.to_datetime(sunrise_down_df['sundown'], format="%H:%M:%S").dt.time
sunrise_down_df.head(5)

,Date,sunrise,sundown
0,2021-01-01,08:47:00,16:37:00
1,2021-01-02,08:47:00,16:38:00
2,2021-01-03,08:46:00,16:39:00
3,2021-01-04,08:46:00,16:41:00
4,2021-01-05,08:46:00,16:42:00


In [20]:
# Merge sun rise/down into sensor data
if not ('sunrise' in sens_df.columns):
    sens_df = pd.merge(sens_df, sunrise_down_df, on ='Date',  how ='left')
    sens_df['DayNight'] = 'night'
    sens_df.loc[(sens_df['Time'] >= sens_df['sunrise']) & (sens_df['Time'] <= sens_df['sundown']),'DayNight'] = 'day'

In [21]:
# Apply correction
sens_df.loc[(sens_df['DayNight'] == 'night'), 'corr_par'] = sens_df.loc[(sens_df['DayNight'] == 'night'), 'Counts'] * led_A + led_B
sens_df.loc[(sens_df['DayNight'] == 'day'), 'corr_par'] = sens_df.loc[(sens_df['DayNight'] == 'day'), 'Counts'] * sun_A + sun_B
sens_df.head(5)

,date,Counts,Date,Time,sunrise,sundown,DayNight,corr_par
0,1900-01-01 09:29:47,0.164062,1900-01-01,09:29:47,NaN,NaN,night,14.936406
1,1900-01-01 09:30:00,0.164062,1900-01-01,09:30:00,NaN,NaN,night,14.936406
2,1900-01-01 09:31:00,0.164062,1900-01-01,09:31:00,NaN,NaN,night,14.936406
3,1900-01-01 09:32:00,0.226562,1900-01-01,09:32:00,NaN,NaN,night,18.382656
4,1900-01-01 09:33:00,0.164062,1900-01-01,09:33:00,NaN,NaN,night,14.936406


In [22]:
# Save results
sens_df.to_csv(f'data/sensor_corrected.csv', sep=',', index=False)

In [23]:
# Plot
fig = go.Figure()
fig.add_trace(go.Scatter(x = sens_df["date"], y=sens_df["Counts"], mode = 'lines', name = 'par data'))
fig.add_trace(go.Scatter(x = sens_df["date"], y=sens_df["corr_par"], mode = 'lines', name = 'corrected par data'))

fig.update_layout(title='Par plot',
                  xaxis_title='time',
                  yaxis_title='umol/(m^2*s)')

# Interactivity
fig.update_layout(
        hovermode="x unified", #show trace line
        xaxis=dict( # Buttons at the top
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="day", step="day",stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider = dict(visible=True), type="date")
)

fig.show()